In [1]:
import os
import pathlib
import numpy as np


import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print("GPU Name: ", gpu_devices)
print("Is TF built with CUDA: ", tf.test.is_built_with_cuda())

GPU Name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is TF built with CUDA:  True


In [3]:
%matplotlib inline
sns.set_style("whitegrid")

## Import Data

In [4]:
DATA_PATH = "./data/pokemon_data/"
data_dir = pathlib.Path(DATA_PATH)

### Number of total images.

In [5]:
image_count = len(list(data_dir.glob('*/*.jpeg')))
image_count

6655

### Number of classes.

In [6]:
class_names = [item.name.lower() for item in data_dir.glob('*')]
len(class_names)

149

### Dictionary for class <=> id conversion

In [7]:
class2idx = {v:k for k,v in enumerate(class_names)}
idx2class = {v:k for k,v in class2idx.items()}

### Paths for images and labels

In [8]:
images = []
labels = []

for class_name in os.listdir(DATA_PATH):
    for img in os.listdir(DATA_PATH + class_name + "/"):
        if img.endswith(".jpeg") or img.endswith(".png"):
            images.append(DATA_PATH + class_name + "/" + img)
            labels.append(class2idx[class_name.lower()])

## Create Dataset

In [9]:
IMG_SIZE = 160
BATCH_SIZE = 8
SHUFFLE_BUFFER = 100

In [10]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify = labels)

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [12]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])


def pre_process_data(file_path, label):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [13]:
train_dataset = train_dataset.map(pre_process_data)
test_dataset = test_dataset.map(pre_process_data)

In [14]:
print("Train dataset: ", train_dataset)
print("Test dataset: ", test_dataset)

Train dataset:  <MapDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>
Test dataset:  <MapDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>


### Exploratory Analysis

In [15]:
def get_class_distribution(dataset_obj):
    count_dict = {k:0 for k,v in class2idx.items()}
    
    for _, label_id in dataset_obj:
        label = idx2class[label_id.numpy()]
        count_dict[label] += 1
    return count_dict

### Class distribution

In [16]:
# get_class_distribution(train_dataset)

## Train Model

### Shuffle and Batch Input Tensors

In [17]:
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(1)

In [18]:
train_dataset

<BatchDataset shapes: ((8, 160, 160, 3), (8,)), types: (tf.float32, tf.int32)>

### Train Classifier Head of PreTrained Model

In [19]:
num_epochs = 10

In [20]:
with tf.device("GPU:0"):

    # Create the base model from the pre-trained model MobileNet V2
    base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')

    # Freeze the pre-trained model weights
    base_model.trainable = False# Trainable classification head

    maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()

    prediction_layer = tf.keras.layers.Dense(149, activation='softmax')# Layer classification head with feature detector

    model = tf.keras.Sequential([
        base_model,
        maxpool_layer,
        prediction_layer
    ])

    learning_rate = 0.0001# Compile the model

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=learning_rate), 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    model.fit(
        train_dataset,
        epochs=num_epochs,
    )

Epoch 1/10
677/677 [==============================] - 20s 29ms/step - loss: 6.5018 - accuracy: 0.0541
Epoch 2/10
677/677 [==============================] - 20s 29ms/step - loss: 3.4095 - accuracy: 0.2805
Epoch 3/10
677/677 [==============================] - 20s 29ms/step - loss: 2.0407 - accuracy: 0.5157
Epoch 4/10
677/677 [==============================] - 20s 29ms/step - loss: 1.3099 - accuracy: 0.6780
Epoch 5/10
677/677 [==============================] - 20s 29ms/step - loss: 0.8828 - accuracy: 0.7903
Epoch 6/10
677/677 [==============================] - 20s 30ms/step - loss: 0.6035 - accuracy: 0.8737
Epoch 7/10
677/677 [==============================] - 20s 29ms/step - loss: 0.4121 - accuracy: 0.9260
Epoch 8/10
677/677 [==============================] - 20s 29ms/step - loss: 0.2829 - accuracy: 0.9577
Epoch 9/10
677/677 [==============================] - 20s 29ms/step - loss: 0.1912 - accuracy: 0.9790
Epoch 10/10
677/677 [==============================] - 20s 29ms/step - loss: 0.130

In [21]:
model.evaluate(test_dataset)

1355/1355 [==============================] - 6s 4ms/step - loss: 1.5236 - accuracy: 0.6376


[1.5235544443130493, 0.6376383900642395]

### Fine Tune PreTrained Model

In [22]:
# Unfreeze all layers of MobileNetV2
base_model.trainable = True

# Refreeze layers until the layers we want to fine-tune
for layer in base_model.layers[:100]:
    layer.trainable =  False
    
    
# Use a lower learning rate
lr_finetune = learning_rate / 10

# Recompile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=lr_finetune), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)


# Increase training epochs for fine-tuning
fine_tune_epochs = 30

total_epochs =  num_epochs + fine_tune_epochs

# Fine-tune model
# Note: Set initial_epoch to begin training after epoch 30 since we
# previously trained for 30 epochs.
model.fit(train_dataset, 
          epochs=total_epochs, 
          initial_epoch = num_epochs
         )

Epoch 11/40
677/677 [==============================] - 23s 34ms/step - loss: 2.2586 - accuracy: 0.4943
Epoch 12/40
677/677 [==============================] - 23s 34ms/step - loss: 1.3818 - accuracy: 0.6377
Epoch 13/40
677/677 [==============================] - 23s 34ms/step - loss: 1.0808 - accuracy: 0.7138
Epoch 14/40
677/677 [==============================] - 24s 35ms/step - loss: 0.8169 - accuracy: 0.7666
Epoch 15/40
677/677 [==============================] - 23s 34ms/step - loss: 0.6892 - accuracy: 0.7965
Epoch 16/40
677/677 [==============================] - 23s 34ms/step - loss: 0.5620 - accuracy: 0.8359
Epoch 17/40
677/677 [==============================] - 23s 34ms/step - loss: 0.4670 - accuracy: 0.8588
Epoch 18/40
677/677 [==============================] - 23s 34ms/step - loss: 0.3824 - accuracy: 0.8852
Epoch 19/40
677/677 [==============================] - 23s 35ms/step - loss: 0.3226 - accuracy: 0.9018
Epoch 20/40
677/677 [==============================] - 24s 35ms/step - lo

In [23]:
model.evaluate(test_dataset)

1355/1355 [==============================] - 6s 4ms/step - loss: 0.8356 - accuracy: 0.8089


[0.8355814814567566, 0.8088560700416565]

In [24]:
y_pred = model.predict(test_dataset)

In [25]:
y_pred_list = []

for i in range(len(y_pred)):
    class_id = np.argmax(y_pred[i])
    y_pred_list.append(class_id)

In [26]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.56      0.71      0.63         7
           1       0.71      0.45      0.56        11
           2       0.86      0.86      0.86         7
           3       0.80      0.50      0.62         8
           4       0.69      0.90      0.78        10
           5       0.80      0.80      0.80         5
           6       1.00      0.80      0.89        10
           7       1.00      1.00      1.00         9
           8       1.00      0.60      0.75        10
           9       1.00      0.89      0.94         9
          10       0.60      0.67      0.63         9
          11       0.90      1.00      0.95         9
          12       0.64      0.70      0.67        10
          13       1.00      0.71      0.83         7
          14       0.91      0.91      0.91        11
          15       0.86      0.75      0.80         8
          16       0.82      1.00      0.90         9
          17       1.00    

In [29]:
model.save("./models/pokemon_mobilenet_pretrained_new.h5")